# KoBert(데이터 증강 이후)
> "NLP"

- toc:true
- branch: master
- badges: true
- comments: true
- author: 전북대학교 통계학과 이강철
- categories: [python]
- hide :false
- published: true

* 데이터 증강했더니 결과가 말도 안되게 좋게 나옴..... 

* 문서 1개당 데이터 증강 기법을 이용하여 10개씩 증가 시킴

 # GPU 용량확인 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


`런타임 초기화 하고 아래코드 돌려서 gpu용량 확인하고 코드돌리자....

In [ ]:
import torch
import gc
gc.collect()

torch.cuda.empty_cache()

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=271d573a9c70e2a88e7a3a2a58c0ef0d94f02fdb2ad0c98e3da4655a55c83c6f
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Gen RAM Free: 25.7 GB  |     Proc size: 1.2 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


# Colab 환경설정

!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

# github에서 KoBERT 파일을 로드 및 KoBERT모델 불러오기

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-4r3ewpxg/kobert-tokenizer_c82b16b44def42eb8d0e87007fa3446b
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-4r3ewpxg/kobert-tokenizer_c82b16b44def42eb8d0e87007fa3446b
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=c152b7704cfcbfa48a71c2292cd07b7ee81f68d742d4fd17cf3ce6cf956a8569
  Stored in directory: /tmp/pip-ephem-wheel-cache-5jjwadk9/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-wf20wsut
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-wf20wsut
     |████████████████████████████████| 132 kB 2.7 MB/s 
     |████████████████████████████████| 344 kB 19.5 MB/s 
     |████████████████████████████████| 47.3 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 57.6 MB/s 
     |████████████████████████████████| 1.2 MB 51.9 MB/s 
     |████████████████████████████████| 3.8 MB 76.8 MB/s 
     |████████████████████████████████| 596 kB 61.5 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 6.5 MB 49.4 MB/s 
     |████████████████████████████████| 895 kB 44.5 MB/s 
     |████████████████████████████████| 8.6 MB 47.3 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 127 kB 85.9 MB/s 
  Created wheel for kobert: filename=kob

In [ ]:
from kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab  = get_pytorch_kobert_model()

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


#  필요한 라이브러리 불러오기

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# 데이터셋 불러오기

In [ ]:
import pandas as pd
data = pd.read_csv('kobert입력데이터2.csv')

In [ ]:
data.head()

,document,topic,text
0,1,15,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
1,1,15,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
2,1,15,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
3,1,15,존경 지지 주택 전시관 계약 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
4,1,15,존경 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 업체 관계 사...


In [ ]:
data["topic"] = data["topic"]-1

In [ ]:
data.head()

,document,topic,text
0,1,14,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
1,1,14,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
2,1,14,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
3,1,14,존경 지지 주택 전시관 계약 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...
4,1,14,존경 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 업체 관계 사...


In [ ]:
data_list = []
for ques, label in zip(data['text'], data['topic'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

#  입력 데이터셋을 토큰화하기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

## 파라미터 셋팅

In [ ]:
max_len = 200
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

# Train data & Test data





In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

## 토큰화 및 패딩, 정수 인코딩 진행

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

# KoBERT 모델 구현

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=23,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.3).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-6},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3999 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.1852335929870605 train acc 0.0625
epoch 1 batch id 201 loss 3.1079132556915283 train acc 0.055970149253731345
epoch 1 batch id 401 loss 2.9384312629699707 train acc 0.08354114713216958
epoch 1 batch id 601 loss 2.4579482078552246 train acc 0.19098377703826955
epoch 1 batch id 801 loss 1.7107940912246704 train acc 0.2938514357053683
epoch 1 batch id 1001 loss 1.6819950342178345 train acc 0.36975524475524474
epoch 1 batch id 1201 loss 1.475642442703247 train acc 0.4274302664446295
epoch 1 batch id 1401 loss 1.1943906545639038 train acc 0.47309957173447537
epoch 1 batch id 1601 loss 0.37908223271369934 train acc 0.5094081823860087
epoch 1 batch id 1801 loss 1.336313247680664 train acc 0.5388499444752916
epoch 1 batch id 2001 loss 0.5357224345207214 train acc 0.5646864067966016
epoch 1 batch id 2201 loss 0.5976346135139465 train acc 0.586182417083144
epoch 1 batch id 2401 loss 0.544891893863678 train acc 0.6052035610162433
epoch 1 batch id 2601 loss 0.533860921859

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1000 [00:00<?, ?it/s]

...

epoch 9 test acc 0.9509375


  0%|          | 0/3999 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0445123054087162 train acc 0.96875
epoch 10 batch id 201 loss 0.1403944492340088 train acc 0.9709266169154229
epoch 10 batch id 401 loss 0.0452195405960083 train acc 0.971399625935162
epoch 10 batch id 601 loss 0.04688456282019615 train acc 0.9723897670549085
epoch 10 batch id 801 loss 0.025614922866225243 train acc 0.9720271535580525
epoch 10 batch id 1001 loss 0.07424446940422058 train acc 0.9716221278721279
epoch 10 batch id 1201 loss 0.01959993503987789 train acc 0.9716382181515404
epoch 10 batch id 1401 loss 0.06428100913763046 train acc 0.9718281584582441
epoch 10 batch id 1601 loss 0.04180331155657768 train acc 0.9719901623985009
epoch 10 batch id 1801 loss 0.011722804978489876 train acc 0.9721335369239311
epoch 10 batch id 2001 loss 0.13746821880340576 train acc 0.9721233133433284
epoch 10 batch id 2201 loss 0.05055586248636246 train acc 0.9719161744661517
epoch 10 batch id 2401 loss 0.036505863070487976 train acc 0.9720168679716785
epoch 10 batch id 

  0%|          | 0/1000 [00:00<?, ?it/s]

epoch 10 test acc 0.95165625


In [ ]:
import pandas as pd

In [ ]:
len(train_history)

200

In [ ]:
len(loss_history)

200

In [ ]:
test_history

[0.8896319444444445,
 0.93728125,
 0.9495,
 0.95140625,
 0.954,
 0.95334375,
 0.9529375,
 0.9530625,
 0.9509375,
 0.95165625]

In [ ]:
total_train = pd.DataFrame()

In [ ]:
total_train["train_history"] = train_history
total_train["loss_history"] = loss_history

In [ ]:
total_test = pd.DataFrame()

In [ ]:
total_test["test_history"] = test_history

In [ ]:
total_train.to_csv("데이터증강 train.csv")

In [ ]:
total_test.to_csv("데이터증강 test.csv")